In [ ]:
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/EHF2018-M-ATTACKS%20(1).xlsx'
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/EHF2020-M-ATTACKS.xlsx'
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/IHF2021-MEN-BIDEA_SCORING.xlsx'
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/IHF2023-M-BIDEA_SCORING%20-FullP.xlsx'
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/IHF2023-M-BIDEA_SCORING%20-FullP.xlsx'
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/EHF2020-W-ATTACKS%20(1).xlsx'
!wget 'https://github.com/JoserraSASG/TFM_Repository/raw/main/data/IHF2021-WOMEN-BIDEA_SCORING.xlsx'

--2023-08-30 08:32:58--  https://github.com/JoserraSASG/TFM_Repository/raw/main/EHF2018-M-ATTACKS%20(1).xlsx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JoserraSASG/TFM_Repository/main/EHF2018-M-ATTACKS%20(1).xlsx [following]
--2023-08-30 08:32:59--  https://raw.githubusercontent.com/JoserraSASG/TFM_Repository/main/EHF2018-M-ATTACKS%20(1).xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13482 (13K) [application/octet-stream]
Saving to: ‘EHF2018-M-ATTACKS (1).xlsx’

EHF2018-M-ATTACKS ( 100%[===================>]  13.17K  --.-KB/s    in 0s      

2023-08-30 08:32:59 (82.3 MB/s) - ‘EHF2018-

In [ ]:
import pandas as pd
import numpy as np

dfEHFM2018 = pd.read_excel('EHF2018-M-ATTACKS (1).xlsx')
dfEHFM2020 = pd.read_excel('EHF2020-M-ATTACKS.xlsx')
dfIHFM2021 = pd.read_excel('IHF2021-MEN-BIDEA_SCORING.xlsx', sheet_name='Teams')
dfIHFM2023 = pd.read_excel('IHF2023-M-BIDEA_SCORING -FullP.xlsx', sheet_name='Teams')
dfEHFW2020 = pd.read_excel('EHF2020-W-ATTACKS (1).xlsx', sheet_name='Export')
dfIHFW2021 =  pd.read_excel('IHF2021-WOMEN-BIDEA_SCORING.xlsx', sheet_name= 'Teams')

# Male

In [ ]:

# ETL processes required to make all columns equal

# Remove unnecessary columns
dfEHFM2020 = dfEHFM2020.drop(['Eff+GK', 'Lanz+GK', 'Eff+GK-%TO', 'Eff+GK-TO'], axis=1)
dfIHFM2021 = dfIHFM2021.drop(['Phase'], axis=1)
dfIHFM2023 = dfIHFM2023.drop(['Phase'], axis=1)
# Calculate %TO in dfs where it's missing
dfIHFM2021['%TO'] = dfIHFM2021['TO'] / dfIHFM2021['Attacks'] * 100
dfIHFM2023['%TO'] = dfIHFM2023['TO'] / dfIHFM2023['Attacks'] * 100
# Rename 'TO%' in dfs 2018 and 2020
dfEHFM2018 = dfEHFM2018.rename(columns={'TO%': '%TO'})
dfEHFM2020 = dfEHFM2020.rename(columns={'TO%': '%TO'})
# Calculate 'Goals' in dfs where it's missing
dfEHFM2018['Goals'] = round((dfEHFM2018['Attacks'] * dfEHFM2018['%Eff']/100), 0)
dfEHFM2020['Goals'] = round((dfEHFM2020['Attacks'] * dfEHFM2020['%Eff']/100), 0)
# Create Match No.
dfEHFM2018[['Match No.', 'Match']] = dfEHFM2018['Match'].str.split(' ', n=1, expand=True)
dfEHFM2020[['Match No.', 'Match']] = dfEHFM2020['Match'].str.split(' ', n=1, expand=True)
# Calculate Diff
dfEHFM2018['Diff'] = dfEHFM2018.groupby('Match No.')['Goals'].diff()
dfEHFM2018['Diff'] = dfEHFM2018['Diff'].fillna(-dfEHFM2018['Diff'].shift(-1))
dfEHFM2020['Diff'] = dfEHFM2020.groupby('Match No.')['Goals'].diff()
dfEHFM2020['Diff'] = dfEHFM2020['Diff'].fillna(-dfEHFM2020['Diff'].shift(-1))
# Create a list of columns in the new order
new_column_order = ['Unnamed: 0', 'Match No.', 'Match', 'Team', 'Attacks', '%Eff', '%Lanz', '%GK', 'TO', '%TO', '¿GANA?', 'Goals', 'Diff']
dfEHFM2018 = dfEHFM2018[new_column_order]
dfEHFM2020 = dfEHFM2020[new_column_order]

In [ ]:
from sklearn.model_selection import train_test_split

# Find the intersection of columns across all dataframes
common_columns = dfEHFM2018.columns.intersection(dfEHFM2020.columns).intersection(dfIHFM2021.columns).intersection(dfIHFM2023.columns)

# Only select the common columns in each dataframe
dfEHFM2018 = dfEHFM2018[common_columns]
dfEHFM2020 = dfEHFM2020[common_columns]
dfIHFM2021 = dfIHFM2021[common_columns]
dfIHFM2023 = dfIHFM2023[common_columns]

# Merge the dataframes
dfM = pd.concat([dfEHFM2018, dfEHFM2020, dfIHFM2021, dfIHFM2023], ignore_index=True)

# Split the dataframe into training and test sets
dfMtrain, dfMtest = train_test_split(dfM, test_size=0.3, random_state=15)

In [ ]:
%%script false --no-raise-error
from google.colab import drive
drive.mount('/content/drive')

# Save the DataFrames to Google Drive
dfMtrain.to_excel('/content/drive/My Drive/datos/dfMtrain.xlsx', index=False)
dfMtest.to_excel('/content/drive/My Drive/datos/dfMtest.xlsx', index=False)

# Female

In [ ]:
# ETL processes required to make all columns equal

# Remove unnecessary columns
dfEHFW2020 = dfEHFW2020.drop(['Shots', '%Eff-Maj', '%Eff-Min', '%Eff-Pos', '%Eff-FB', '%Eff-Ind-FB', '%Eff-Team-FB'], axis=1)
dfIHFW2021 = dfIHFW2021.drop(['Match No.'], axis=1)
# Create '%TO' in dfs where it doesn't exist
dfIHFW2021['%TO'] = dfIHFM2021['TO'] / dfIHFM2021['Attacks'] * 100
# Rename columns
dfEHFW2020 = dfEHFW2020.rename(columns={'TO %': '%TO'})
dfEHFW2020 = dfEHFW2020.rename(columns={'Dif': 'Diff'})

In [ ]:
from sklearn.model_selection import train_test_split

# Find the intersection of columns across all dataframes
common_columns = dfEHFW2020.columns.intersection(dfIHFW2021.columns)

# Only select the common columns in each dataframe
dfEHFW2020 = dfEHFW2020[common_columns]
dfIHFW2021 = dfIHFW2021[common_columns]

# Merge the dataframes
dfW = pd.concat([dfEHFW2020, dfIHFW2021], ignore_index=True)

# Split the dataframe into training and test sets
dfWtrain, dfWtest = train_test_split(dfW, test_size=0.3, random_state=15)

In [ ]:
%%script false --no-raise-error
from google.colab import drive
drive.mount('/content/drive')

# Save the DataFrames to Google Drive
dfWtrain.to_excel('/content/drive/My Drive/datos/dfWtrain.xlsx', index=False)
dfWtest.to_excel('/content/drive/My Drive/datos/dfWtest.xlsx', index=False)